# $Importation$ $des$ $bibliothèques$

In [104]:
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
import os
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt
import re
import heapq
from collections import Counter

# $Données/Informations$ $utilisées$ $pour$ $le$ $programme$

In [105]:
path = 'eren_rgb.jpg'
path1 = 'file.txt'
path2 = 'reconstiued3.jpg'
num_rows = 4
num_colors = 16
num_blocs = 16

# $Caratéristiques$ $de$ $l'image$

In [106]:
def details(path):
    img = Image.open(path)
    size = img.size
    format = img.format
    mode = img.mode
    resolution = img.info.get('dpi')
    definition = size[0] * size[1]
    definition2 = size
    
    if resolution is not None:
        size = (size[0]/ resolution[0], size[1] / resolution[1])
    else:
        size = (0, 0)  # or any default resolution you want to use
    
    poids = os.path.getsize(path) / (1000.0)  # Convert to kilobytes
    
    if mode == 'RGB':
        trsc = definition * 3 * img.bits
    else:
        trsc = definition * 1 * img.bits
    
    trsc /= (1024 * 8)
    
    taux_compression = 100 * (1 - (poids/trsc))
        
    return size, format, mode, resolution, definition, definition2, poids, trsc, taux_compression

In [107]:
# Exemple d'utilisation
result = details(path)
print("Size (inch):", result[0])
print("Format:", result[1])
print("Mode:", result[2])
print("Resolution:", result[3])
print("Definition (pixels):", result[4])
print("Définition (L * C):", result[5])
print("Taille en mémoire avec compression:", result[6], "KB")
print("Taille réelle sans compression:", result[7]/1024, "MB")
print("Taux de compression: ", result[8], "%")

Size (inch): (0, 0)
Format: JPEG
Mode: RGB
Resolution: None
Definition (pixels): 49728
Définition (L * C): (224, 222)
Taille en mémoire avec compression: 12.665 KB
Taille réelle sans compression: 0.14227294921875 MB
Taux de compression:  91.3067353067353 %


# $Affichage$ $de$ $la$ $palette$ $de$ $couleurs$ $et$ $des$ $images$ $avant$ $et$ $après$ $application$ $de$ $la$ $palette$ $de$ $couleurs$

In [108]:
def display_image_with_palette(image, color_palette, palette_indices, num_rows):
    num_colors = len(color_palette)
    num_cols = num_colors // num_rows

    image_with_palette = color_palette[palette_indices]

    plt.figure(figsize=(30, 15))
    for i, color in enumerate(color_palette):
        plt.subplot(num_rows, num_cols, i + 1)
        color_patch = np.zeros((100, 100, 3), dtype=np.uint8)
        color_patch[:, :] = color
        plt.imshow(color_patch)
        plt.title(f'Couleur {i+1}')
        plt.axis('off')
    plt.suptitle('Palette de couleurs', fontsize=16)
    plt.tight_layout()
    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(30, 15))
    fig.subplots_adjust(wspace=0.01)

    axs[0].imshow(image)
    axs[0].set_title('Image originale')
    axs[0].axis('off')

    axs[1].imshow(image_with_palette)
    axs[1].set_title(f'Image avec la palette de {num_colors} couleurs')
    axs[1].axis('off')

    fig.tight_layout()

    plt.show()

# $Affichage$ $de$ $deux$ $images$ $quelconques$ $côte$ $à$ $côte$

In [109]:
def plot_images(reconstructed_image, image1, num_colors):
    fig, axs = plt.subplots(1, 2, figsize=(30, 15))
    fig.subplots_adjust(wspace=0.01)

    axs[0].imshow(reconstructed_image)
    axs[0].set_title('Image reconstruite')
    axs[0].axis('off')

    axs[1].imshow(image1)
    axs[1].set_title(f'Image avec la palette de {num_colors} couleurs (median-cut)')
    axs[1].axis('off')

    fig.tight_layout()

    plt.show()

## $Mean$ $Square$ $Error$ $(MSE):$ $Erreur$ $Quadratique$ $Moyenne$

In [110]:
def MSE(image_d_origine, image_traitee):
    # Get the dimensions of the images
    L, C = np.array(image_d_origine).shape[:2]  # Assuming it's a 2D image, getting rows and columns
    image_traitee = np.array(image_traitee).reshape(np.array(image_d_origine).shape)
    # Convert the images to NumPy arrays
    origine = np.array(image_d_origine)
    traitee = np.array(image_traitee)

    # Calculate the sum of squared differences
    somme_differences_carrees = np.sum((origine - traitee) ** 2)

    # Calculate (1 / (L * C)) * somme
    resultat = (1 / (L * C)) * somme_differences_carrees

    return resultat

# $Taux$ $de$ $compression$

In [111]:
def taux_compression(img_path, compressed_file_path):
    img = Image.open(img_path)
    
    if img.mode == 'RGB':
        pixel_size = 3  # Nombre de canaux de couleur (RGB)
    else:
        pixel_size = 1  # Pour d'autres modes de couleur
    
    if img.bits == 8:  # Taille en bits par pixel
        pixel_bits = 8
    else:
        pixel_bits = 16  # Peut être ajusté selon les besoins
    
    img_data_size = img.size[0] * img.size[1] * pixel_size * pixel_bits  # Taille en bits de l'image
    
    # Lecture des données compressées à partir du fichier .irm
    with open(compressed_file_path, 'rb') as f:
        compressed_data = f.read()
    
    compressed_data_size = len(compressed_data) * 8  # Taille des données compressées en bits
    
    compression_ratio = img_data_size / compressed_data_size  # Ratio de compression
    
    return compression_ratio

# (1). $Création$ $de$ $la$ $palette$ $de$ $couleurs$

## (a). $Sens$ $allée$

### $k-means$ $clustering$

In [112]:
def kmeans_clustering_palette(image, num_colors):
    # Convertir l'image en un tableau 2D de pixels
    pixels = np.reshape(image, (-1, 3))  # (nombre de pixels, 3 canaux de couleur)

    # Appliquer l'algorithme de k-means clustering
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters (couleurs dominantes)
    color_palette = kmeans.cluster_centers_.astype(int)

    # Associer chaque pixel à l'indice de couleur dans la palette
    labels = kmeans.predict(pixels)

    # Reformater les indices des couleurs dans la palette selon la forme de l'image originale
    palette_indices = np.reshape(labels, np.array(image).shape[:2])

    return color_palette, palette_indices

### $Median-cut$

In [113]:
def median_cut_palette(image, num_colors):
    # Convertir l'image en un tableau 2D de pixels
    pixels = np.reshape(image, (-1, 3))

    # Initialiser la liste des cubes de couleur avec le cube contenant tous les pixels
    cubes = [pixels]

    # Répéter jusqu'à ce que le nombre de cubes atteigne le nombre de couleurs souhaité
    while len(cubes) < num_colors:
        # Sélectionner le cube le plus grand
        largest_cube_index = np.argmax([cube.shape[0] for cube in cubes])
        largest_cube = cubes.pop(largest_cube_index)

        # Trouver l'axe dominant pour diviser le cube
        axis = np.argmax(np.max(largest_cube, axis=0) - np.min(largest_cube, axis=0))

        # Trier les pixels du cube le long de l'axe dominant
        sorted_cube = largest_cube[largest_cube[:, axis].argsort()]

        # Diviser le cube en deux parties égales
        split_index = len(sorted_cube) // 2
        cube1 = sorted_cube[:split_index]
        cube2 = sorted_cube[split_index:]

        # Ajouter les deux nouveaux cubes à la liste
        cubes.append(cube1)
        cubes.append(cube2)

    # Calculer les couleurs moyennes pour chaque cube
    color_palette = [np.mean(cube, axis=0) for cube in cubes]
    color_palette = np.array(color_palette).astype(np.uint8)

    # Calculer les indices des couleurs dans la palette pour chaque pixel de l'image
    palette_indices = np.zeros(len(pixels), dtype=int)
    for i, pixel in enumerate(pixels):
        distances = np.linalg.norm(color_palette - pixel, axis=1)
        palette_indices[i] = np.argmin(distances)

    palette_indices = np.reshape(palette_indices, np.array(image).shape[:2])

    return color_palette, palette_indices

### $Octree$

In [114]:
def octree_palette(image, num_colors):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
        
    img_rgb = image.convert("RGB")
    img_with_palette = img_rgb.quantize(colors=num_colors, method=Image.FASTOCTREE)
    palette = img_with_palette.getpalette()[:num_colors * 3]
    palette = np.array(palette).reshape(-1, 3)
    indices = np.array(img_with_palette)

    # Retourner la palette de couleurs et leurs indices
    return palette, indices

## (b). $Sens$ $retour$

### $k-means$ $clustering$

In [115]:
def inverse_kmeans_clustering_palette(color_palette, palette_indices):
    # Récupérer les dimensions de l'image à partir des indices de palette
    height, width = palette_indices.shape

    # Initialiser une image vide avec les dimensions récupérées
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Remplir l'image reconstruite avec les couleurs de la palette
    for i in range(height):
        for j in range(width):
            color_index = palette_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]

    return reconstructed_image

### $Median-cut$

In [116]:
def inverse_median_cut_palette(color_palette, palette_indices):
    # Récupérer les dimensions de l'image à partir des indices de palette
    height, width = palette_indices.shape

    # Initialiser une image vide avec les dimensions récupérées
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Remplir l'image reconstruite avec les couleurs de la palette
    for i in range(height):
        for j in range(width):
            color_index = palette_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]
  
    return reconstructed_image

### $Octree$

In [117]:
def inverse_octree_palette(color_palette, palette_indices):
    color_palette = color_palette.reshape(-1)
    color_palette = [(int(color_palette[i]), int(color_palette[i + 1]), int(color_palette[i + 2])) for i in range(0, len(color_palette), 3)]
    reconstructed_image = Image.new("RGB", (palette_indices.shape[1], palette_indices.shape[0]))
    reconstructed_image.putdata([color_palette[idx] for idx in palette_indices.flatten()])
  
    return reconstructed_image

# (2). $Mappage$ $des$ $pixels$

## (a). $Sens$ $"allée"$

In [118]:
def map_to_palette(image, color_palette):
    # Assurez-vous que l'image est sous forme de tableau numpy
    image = np.array(image)
    
    # Redimensionner les pixels en une matrice (nombre de pixels, 3 canaux de couleur)
    pixels = np.reshape(image, (-1, 3))  

    # Calculer la distance de chaque pixel à chaque couleur de la palette
    distances = np.linalg.norm(pixels[:, np.newaxis] - color_palette, axis=2)

    # Obtenir l'indice de la couleur la plus proche pour chaque pixel
    indices = np.argmin(distances, axis=1)

    # Reformater les indices des couleurs selon la forme de l'image originale
    mapped_indices = np.reshape(indices, image.shape[:2])

    return np.array(mapped_indices)

## (b). $Sens$ $"retour"$

In [119]:
def inverse_map_to_palette(mapped_indices, color_palette):
    # Obtenir les dimensions de l'image reconstruite
    height, width = mapped_indices.shape

    # Créer une image vide
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Pour chaque pixel, assigner la couleur correspondante de la palette
    for i in range(height):
        for j in range(width):
            color_index = mapped_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]

    return Image.fromarray(reconstructed_image)

# (3). $Subdivision$ $en$ $blocs$

## (a). $Sens$ $"allée"$

In [120]:
def subdivision2D(matrice, taille_blocs):
    # Dimensions de la matrice initiale
    lignes, colonnes = matrice.shape
    
    # Dimensions des blocs
    blocs_lignes, blocs_colonnes = (taille_blocs, taille_blocs)
    
    # Calcul des dimensions des blocs avec zéros ajoutés
    new_blocs_lignes = (lignes + blocs_lignes - 1) // blocs_lignes
    new_blocs_colonnes = (colonnes + blocs_colonnes - 1) // blocs_colonnes
    
    # Initialisation de la matrice des sous-matrices
    sous_matrices = np.zeros((new_blocs_lignes, new_blocs_colonnes, blocs_lignes, blocs_colonnes))
    
    # Remplissage de la matrice des sous-matrices
    for i in range(new_blocs_lignes):
        for j in range(new_blocs_colonnes):
            sous_matrices[i, j, :min(blocs_lignes, lignes - i*blocs_lignes), :min(blocs_colonnes, colonnes - j*blocs_colonnes)] = \
                matrice[i*blocs_lignes:(i+1)*blocs_lignes, j*blocs_colonnes:(j+1)*blocs_colonnes]
    
    # Convertir la matrice de sous-matrices en une liste de matrices 2D
    liste_matrices = []
    for i in range(new_blocs_lignes):
        for j in range(new_blocs_colonnes):
            liste_matrices.append(sous_matrices[i, j])
    
    return liste_matrices, (lignes, colonnes)

## (b). $Sens$ $"retour"$

In [121]:
def reconstitution2D(liste_matrices, dimensions):# Récupération des dimensions de la matrice initiale
    lignes, colonnes = dimensions
    
    # Initialisation de la matrice résultante avec des zéros
    matrice_resultante = np.zeros((lignes, colonnes))
    
    # Dimensions des blocs
    taille_blocs = liste_matrices[0].shape
    
    # Nombre de blocs
    nb_blocs_lignes = (lignes + taille_blocs[0] - 1) // taille_blocs[0]
    nb_blocs_colonnes = (colonnes + taille_blocs[1] - 1) // taille_blocs[1]
    
    # Recombinaison des sous-matrices dans la matrice résultante
    for i in range(nb_blocs_lignes):
        for j in range(nb_blocs_colonnes):
            matrice_resultante[i*taille_blocs[0]:(i+1)*taille_blocs[0], j*taille_blocs[1]:(j+1)*taille_blocs[1]] = \
                liste_matrices[i*nb_blocs_colonnes + j][:min(taille_blocs[0], lignes - i*taille_blocs[0]), :min(taille_blocs[1], colonnes - j*taille_blocs[1])]
    
    return matrice_resultante

# (4). $Sous$ $échantillonnage$

## (a). $Sens$ $"allée"$

### $4:2:0$

In [122]:
def sous_echantillonnage_4_2_0(image):
    # Sous-échantillonnage de l'image
    sub_image = image[::2, ::2]
    
    return sub_image

### $4:2:2$

In [123]:
def sous_echantillonnage_4_2_2(image):
    # Sous-échantillonnage de l'image
    sub_image = image[::1, ::2]
    
    return sub_image

### $4:4:4$

In [124]:
def sous_echantillonnage_4_4_4(image):
    # La fonction ne fait rien car il n'y a pas de sous-échantillonnage pour le 4:4:4
    return image.copy() 

## (b). $Sens$ $"retour"$

### $4:2:0$

In [125]:
def inverse_sous_echantillonnage_4_2_0(imSub):
    # Répétition des pixels pour restaurer la résolution horizontale
    H, W = imSub.shape
    res_h = np.repeat(imSub, 2, axis=1)
    
    # Répétition des pixels pour restaurer la résolution verticale
    res_v = np.repeat(res_h, 2, axis=0)
    
    return res_v

### $4:2:2$

In [126]:
def inverse_sous_echantillonnage_4_2_2(imSub):
    # Répétition des colonnes pour restaurer les canaux de chrominance
    res_h = np.repeat(imSub, 2, axis=1)
    
    return res_h

### $4:4:4$

In [127]:
def inverse_sous_echantillonnage_4_4_4(imSub):
    # L'inverse du sous-échantillonnage 4:4:4 est simplement une copie de l'image d'entrée
    return imSub.copy()

# (5). $Transformée$ $en$ $cosinus$ $discrète:$ $application$ $de$ $la$ $DCT$

## (a). $Sens$ $"allée"$

In [128]:
def apply_dct(matrix):
    # Appliquer la DCT à chaque matrice
    dct_matrix = dct(dct(matrix.T, norm='ortho').T, norm='ortho')
    
    return np.array(dct_matrix)

## (b). $Sens$ $"retour"$

In [129]:
def inverse_dct(dct_matrix):
    # Appliquer l'inverse de la DCT à chaque matrice
    inverse_dct_matrix = idct(idct(dct_matrix.T, norm='ortho').T, norm='ortho')
    
    return np.array(inverse_dct_matrix)

# (6). $Quantification$

## (a). $Sens$ $"allée"$

### $Valeur$ $minimale$

In [130]:
def quantization_min(matrix):
    # Déterminer la valeur minimale dans la matrice
    min_value = np.min(matrix)
    
    # Vérifier si la valeur minimale est proche de zéro avant la division
    if np.abs(min_value) == 0:
        min_value = 1
        # Quantification en utilisant la valeur minimale comme facteur de quantification
        quantized_dct = np.round(matrix)
    else:
        # Division par epsilon si la valeur minimale est proche de zéro
        quantized_dct = np.round(matrix / min_value)
    
    if np.all(quantized_dct == 0):
        quantized_dct[0][0] = 1
    
    return quantized_dct, min_value

### $Valeur$ $moyenne$

In [131]:
def quantization_mean(matrix):
    # Déterminer la valeur minimale et maximale dans la matrice
    min_value = np.min(matrix)
    max_value = np.max(matrix)
    
    # Calculer la moyenne des valeurs
    mean_value = (max_value + min_value) / 2
    
    # Vérifier si la valeur moyenne est proche de zéro avant la division
    if np.abs(mean_value) == 0:
        mean_value = 1
        # Quantification en utilisant la valeur moyenne comme facteur de quantification
        quantized_dct = np.round(matrix)
    else:
        # Division par epsilon si la valeur moyenne est proche de zéro
        quantized_dct = np.round(matrix / mean_value)
    
    if np.all(quantized_dct == 0):
        quantized_dct[0][0] = 1
    
    return quantized_dct, mean_value

### $Valeur$ $maximale$

In [132]:
def quantization_max(matrix):
    # Déterminer la valeur maximale dans la matrice
    max_value = np.max(matrix)
    
    # Vérifier si la valeur maximale est proche de zéro avant la division
    if np.abs(max_value) == 0:
        max_value = 1
        # Quantification en utilisant la valeur maximale comme facteur de quantification
        quantized_dct = np.round(matrix)
    else:
        # Division par epsilon si la valeur maximale est proche de zéro
        quantized_dct = np.round(matrix / max_value)
    
    if np.all(quantized_dct == 0):
        quantized_dct[0][0] = 1
    
    return quantized_dct, max_value

## (b). $Sens$ $"retour"$

### $Valeur$ $minimale$

In [133]:
def dequantization_min(quantized_matrix, min_value):
    return np.array(quantized_matrix * min_value, dtype=np.float64)

### $Vaeur$ $moyenne$

In [134]:
def dequantization_mean(quantized_matrix, mean_value):
    return np.array(quantized_matrix * mean_value, dtype=np.float64)

### $Valeur$ $maximale$

In [135]:
def dequantization_max(quantized_matrix, max_value):
    return np.array(quantized_matrix * max_value, dtype=np.float64)

# (7). $Vectorisation$

## (a). $Sens$ $"allée"$

### $En$ $ligne$

In [136]:
def ligne_scan(matrice):
  shape = matrice.shape
  vecteur = matrice.flatten()
  return np.array(vecteur, dtype=np.int8), shape

### $En$ $colonne$

In [137]:
def colonne_scan(matrice):
    shape = matrice.shape
    vecteur = np.ravel(matrice, order = 'F')
    return np.array(vecteur, dtype=np.int8), shape

### $En$ $zigzag$

In [138]:
def zigzag_scan(matrice):
  shape = matrice.shape
  vecteur = np.concatenate([np.diagonal(matrice[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - matrice.shape[0], matrice.shape[0])])
  return np.array(vecteur, dtype=np.int8), shape

## (b). $Sens$ $"retour"$

### $En$ $ligne$

In [139]:
def inverse_ligne_scan(vecteur, shape):
  mat = np.array(vecteur).reshape((shape[0], shape[1]))
  return mat

### $En$ $colonne$

In [140]:
def inverse_colonne_scan(vecteur, shape):
    mat = np.array(vecteur).reshape((shape[0], shape[1]))
    return mat.T

### $En$ $zigzag$

In [141]:
def inverse_zigzag_scan(vector, shape):
    rows, cols = shape
    mat = np.array([[None] * cols for _ in range(rows)])
    row, col = 0, 0
    direction = 1

    for i in range(rows * cols):
        mat[row][col] = vector[i]
        if direction == 1:
            if col == cols - 1:
                row += 1
                direction = -1
            elif row == 0:
                col += 1
                direction = -1
            else:
                row -= 1
                col += 1
        else:
            if row == rows - 1:
                col += 1
                direction = 1
            elif col == 0:
                row += 1
                direction = 1
            else:
                row += 1
                col -= 1

    #return np.array(mat, dtype=np.uint8)
    return mat

# (8). $Codage$ $de$ $Huffman$

## (-). $Fonctions$ $intermédiaires$ $(propres$ $à$ $Huffman)$

### $Extraction$ $des$ $caractères$ $et$ $leur$ $fréquence$ $d'apparition$

In [142]:
def ext_chr(chaine):
    occ = {c: chaine.count(c) for c in set(chaine)}
    return sorted(occ.items(), key=lambda x: x[1], reverse=True)

### $Dictionnaire$ $de$ $Huffman$

In [143]:
def huffman_dictionnary_code(node, binString=''):
    if isinstance(node, str):
        return {node: binString}
    (l, r) = node
    d = {}
    d.update(huffman_dictionnary_code(l, binString + '0'))
    d.update(huffman_dictionnary_code(r, binString + '1'))
    return d

### $Arbre$ $de$ $Huffman$

In [144]:
def huffman_tree(chaine):
    nodes = ext_chr(chaine)
    
    while len(nodes) > 1:
        (key1, c1) = nodes.pop()
        (key2, c2) = nodes.pop()
        node = (key1, key2)
        nodes.append((node, c1 + c2))
        nodes.sort(key=lambda x: x[1], reverse=True)
        
    return nodes[0][0]

## (-). $Fonctions$ $intermédiaires$ $(impropres)$

### $Convertion$ $du$ $type$ $des$ $valeurs$ $du$ $vecteur$ $en$ $entier$ $non$ $signé$ $sur$ $8$ $bits$ $et$ $convertion$ $en$ $une$ $chaine$ $de$ $caractères$

#### (a). $Sens$ $"allée"$

In [145]:
def rle_encode(data):
    encoded_data = ""
    current_char = data[0]
    count = 1

    for i in range(1, len(data)):
        if data[i] == current_char:
            count += 1
        else:
            if count > 1:  # Ajout de cette condition
                encoded_data += str(current_char) + "_" + str(count) + "*"
            else:
                encoded_data += str(current_char) + "*"
            current_char = data[i]
            count = 1

    if count > 1:  # Ajout de cette condition
        encoded_data += str(current_char) + "_" + str(count)
    else:
        encoded_data += str(current_char)

    return encoded_data

In [146]:
def rle_decode(encoded_data):
    decoded_data = []
    encoded_data_split = encoded_data.split('*')

    for item in encoded_data_split:
        if "_" in item:
            char, count = item.split("_")
            char = float(char)
            if char.is_integer():
                char = int(char)
            decoded_data.extend([char] * int(count))
        else:
            char = float(item)
            if char.is_integer():
                char = int(char)
            decoded_data.append(char)

    return decoded_data

In [147]:
def convert_and_concatenate(vector):
    # Conversion du vecteur en une liste de nombres non signés
    unsigned_vector = vector.astype(np.uint8) 
    
    # Obtention de la taille de chaque élément de la liste
    element_size = [len(str(i)) for i in unsigned_vector.tolist()]

    # Concaténation des nombres non signés en une liste unique
    concatenated_list = unsigned_vector.flatten().tolist()
    string = ''.join(map(str, concatenated_list))

    return string, element_size

#### (b). $Sens$ $"retour"$

In [148]:
def inverse_convert_and_concatenate(string, element_size):
    # Vérification que les arguments sont valides
    if not isinstance(string, str) or not isinstance(element_size, list):
        raise ValueError("Les arguments doivent être une chaîne de caractères et une liste.")

    # Convertir la chaîne en une liste de nombres non signés en utilisant les tailles d'éléments
    unsigned_vector = []
    start = 0
    for size in element_size:
        unsigned_vector.append(int(string[start:start+size]))
        start += size

    # Convertir la liste de nombres non signés en vecteur numpy
    vector = np.array(unsigned_vector, dtype=np.int8)

    return vector

## (a). $Codage$

In [149]:
def encoding_huffman(strings):
    huffman_tree_result = huffman_tree(strings)  # Stockez l'arbre de Huffman dans une variable distincte
    huffmanCode = huffman_dictionnary_code(huffman_tree_result)
    compressed_string = ''
    
    for char in strings:
        compressed_string += huffmanCode[char]
        
    return compressed_string, huffman_tree_result


## (b). $Décodage$

In [150]:
def decoding_huffman(compressed_string, huffman_tree):
    decoded_string = ''
    current_node = huffman_tree
    
    for bit in compressed_string:
        if bit == '0':
            current_node = current_node[0]
        else:
            current_node = current_node[1]
        
        if isinstance(current_node, str):
            decoded_string += current_node
            current_node = huffman_tree
    
    return decoded_string

## (-). $Fonctions$ $de$ $codage$ $du$ $vecteur$ $à$ $Huffman$

### $Codage$

In [151]:
def huffman_concatenated(vect):
  string, vect_lengths = convert_and_concatenate(vect)
  huffman_code, huffman_tr = encoding_huffman(string)

  return huffman_code, huffman_tr, vect_lengths

### $Décodage$

In [152]:
def huffman_deconcatenated(huffman_code, huffman_tr, vect_lengths):
    decoded = decoding_huffman(huffman_code, huffman_tr)
    vect = inverse_convert_and_concatenate(decoded, vect_lengths)
    
    return vect

# (9). $Codage$ $par$ $LZW$

## (-). $Fonctions$ $intermediaires$

### $Fusion$ $de$ $la$ $liste$ $des$ $codes$ $Huffman$ $de$ $tous$ $les$ $blocs$

In [153]:
def fusionner(strings_list):
    tailles = []
    string_fusionned = ""
    
    # Parcourir tous les éléments de la liste
    for string in strings_list:
        # Convertir chaque nombre en chaîne de caractères et l'ajouter à la chaîne fusionnée
        string_fusionned += string
        
        # Ajouter la taille de l'élément à la liste des tailles
        tailles.append(len(string))
    
    # Retourner la chaîne de caractères fusionnée et la liste des tailles
    return string_fusionned, tailles

### $Séparation$ $de$ $la$ $liste$ $des$ $codes$ $Huffman$ $de$ $tous$ $les$ $blocs$

In [154]:
def separer(string_fusionned, tailles):
    strings_list = []
    debut = 0
    
    # Parcourir toutes les tailles dans la liste
    for taille in tailles:
        # Extraire le sous-chaîne correspondant à la taille actuelle
        sous_chaine = string_fusionned[debut:debut+taille]
        
        # Convertir la sous-chaîne en nombre et l'ajouter à la liste des nombres
        strings_list.append(sous_chaine)
        
        # Mettre à jour la position de départ pour la prochaine sous-chaîne
        debut += taille
    
    # Retourner la liste des nombres
    return strings_list

## (a). $Codage$

In [155]:
def encoding_lzw(data, color_palette):
    alphabet = [str(i) for i in range(len(color_palette))]
    data_fused, taille = fusionner(data)  # Fusionner les chaînes de caractères
    encoded_data = []

    dictionary = {}  # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[char] = i

    prefix = ''
    for char in data_fused:
        new_entry = prefix + char
        if new_entry in dictionary:
            prefix = new_entry
        else:
            encoded_data.append(dictionary[prefix])
            dictionary[new_entry] = len(dictionary)
            prefix = char

    if prefix:
        encoded_data.append(dictionary[prefix])

    return encoded_data, alphabet, taille

## (b). $Décodage$

In [156]:
def decoding_lzw(compressed_data, alphabet, tailleL):
    result = []
    dictionary = {}
    current_code = len(alphabet)

    # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[i] = char

    old_entry = dictionary[compressed_data[0]]
    result.append(old_entry)
    for new_entry in compressed_data[1:]:
        if new_entry in dictionary:
            entry = dictionary[new_entry]
        elif new_entry == current_code:
            entry = old_entry + old_entry[0]
        else:
            raise ValueError("Mauvaise séquence compressée")

        result.append(entry)

        # Utilisez le même dictionnaire pour la décompression
        dictionary[current_code] = old_entry + entry[0]
        current_code += 1
        old_entry = entry

    result = ''.join(result)
    result = separer(result, tailleL)
    
    return result

## $Fonction$ $intermediaire$ $pour$ $la$ $compression$

In [157]:
def intermediaireC(imageA, num_colorsA):
  color_paletteA, palette_indicesA = kmeans_clustering_palette(imageA, num_colorsA)
  mapped_indicesA = map_to_palette(imageA, color_paletteA)
  blocsA, shapeTA = subdivision2D(mapped_indicesA, num_blocs)
  dataA = []
  huffman_trsA = []
  vect_lengthssA = []
  
  for blocA in blocsA:
    result1A = sous_echantillonnage_4_2_0(blocA)
    dct_matrixA = apply_dct(result1A)
    result11A, mean_valueA = quantization_mean(dct_matrixA)
    ligneA, shapetA = ligne_scan(result11A)
    rleA = rle_encode(ligneA)
    huffman_codeA, huffman_trA = encoding_huffman(rleA)
    
    if not huffman_codeA:
      print(ligneA)
      print(result11A)
      
    huffman_trsA.append(huffman_trA)
    dataA.append(huffman_codeA)
  
  encoded_dataA, alphabetA, tailleA = encoding_lzw(dataA, color_paletteA)
    
  return encoded_dataA, alphabetA, tailleA, huffman_trsA, shapeTA, shapetA, mean_valueA, mapped_indicesA, color_paletteA, palette_indicesA

## $Fonction$ $intermediaire$ $pour$ $la$ $décompression$

In [168]:
def intermediaireD(encoded_dataB, alphabetB, tailleB, huffman_trB, shapeTB, shapetB, mean_valueB, mapped_indicesB, color_paletteB, palette_indicesB):
  resultB = decoding_lzw(encoded_dataB, alphabetB, tailleB)
  codesB = []

  for i in range(len(resultB)):
    vect1B = decoding_huffman(resultB[i], huffman_trB[i])
    rleB = rle_decode(vect1B)
    matB = inverse_ligne_scan(rleB, shapetB)
    quantB = dequantization_mean(matB, mean_valueB)
    dctB = inverse_dct(quantB)
    subsapB = inverse_sous_echantillonnage_4_2_0(dctB)
    codesB.append(subsapB)
  
  code7 = reconstitution2D(codesB, shapeTB)
  mapp = inverse_map_to_palette(mapped_indicesB, color_paletteB)
  imageB = inverse_kmeans_clustering_palette(color_paletteB, palette_indicesB)
  return imageB

## $Fonction$ $intermediaire$ $pour$ $la$ $création$ $de$ $la$ $partie$ $"header"$

In [159]:
def create_header(path, num_colors):
  image = Image.open(path)
  return intermediaireC(image, num_colors)[1:]

## $Création$ $de$ $la$ $partie$ $"data"$ $(donnée)$

In [160]:
def data_irm(path, num_colors):
  image = Image.open(path)
  return intermediaireC(image, num_colors)[0]

## $Création$ $de$ $la$ $partie$ $"header"$ $(entête)$

In [161]:
def header_irm(path, num_colors):
    alphabet, taille, huffman_trT, shapeT, shapet, mean_value, mapped_indices, color_palette, palette_indices = create_header(path, num_colors)
    header = f"alphabet: {alphabet}\n"
    header += f"taille: {taille}\n"
    header += f"huffman_trT: {huffman_trT}\n"
    header += f"shapeT: {shapeT}\n"
    header += f"shapet: {shapet}\n"
    header += f"mean_value: {mean_value}\n"
    header += f"mapped_indices: {list(map(list,list(mapped_indices)))}\n"
    header += f"color_palette: {list(map(list,list(color_palette)))}\n"
    header += f"palette_indices: {list(map(list,list(palette_indices)))}\n"
    return header

## $Fonction$ $de$ $compression$

In [162]:
def compression_irm(path1, num_colors, path2):
    header = header_irm(path1, num_colors)
    data = data_irm(path1, num_colors)
    compressed_data = "\n".join(str(nombre) for nombre in data)
    with open(path2, 'w') as f:
        f.write(header)
        f.write(compressed_data)

## $Fonction$ $de$ $décompression$

In [169]:
def decompression_irm(path1, path2):
    with open(path1, 'r') as f:
        lines = f.readlines()
        numeric_lines = [line.strip() for line in lines if re.match(r'^\d+$', line.strip())]
        encoded_data = [int(nombre.strip()) for nombre in numeric_lines]
        alphabet = np.array(eval(lines[0].split(': ')[1]))
        taille = np.array(eval(lines[1].split(': ')[1]))
        huffman_trT = eval(lines[2].split(': ')[1])
        shapeT = eval(lines[3].split(': ')[1])
        shapet = eval(lines[4].split(': ')[1])
        mean_value = eval(lines[5].split(': ')[1]) 
        mapped_indices = np.array(eval(lines[6].split(': ')[1]))
        color_palette = np.array(eval(lines[7].split(': ')[1]))
        palette_indices = np.array(eval(lines[8].split(': ')[1]))
    
    image = intermediaireD(encoded_data, alphabet, taille, huffman_trT, shapeT, shapet, mean_value, mapped_indices, color_palette, palette_indices)
    image.save(path2)
    return image

# $APPLICATION$ $COMPLÈTE$

## $Compression$

In [164]:
compression_irm(path, num_colors, path1)

/home/issa-sangare/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/issa-sangare/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## $Décompression$

In [170]:
decompression_irm(path1, path2)

ValueError: could not convert string to float: ''

## $Affichage$

In [ ]:
img1 = Image.open(path)
img2 = Image.open(path2)

In [ ]:
plot_images(img1, img2)

## $Mean$ $Square$ $Error$ $(MSE):$ $Erreur$ $Quadratique$ $Moyenne$

In [ ]:
erreur = MSE(img1, img2)
print(f"L'erreur quadratique moyenne (MSE) est: {erreur}")

## $Taux$ $de$ $compression$

In [ ]:
taux = taux_compression(path, path2)
print(f"Taux de compression: {100 * taux} %")

FileNotFoundError: [Errno 2] No such file or directory: 'reconstiued3.jpg'

In [13]:
import numpy as np

def duplicate_border(matrix):
    # Récupération des dimensions de la matrice d'origine
    rows, cols = matrix.shape
    
    # Création d'une nouvelle matrice avec des dimensions augmentées pour accueillir les bords dupliqués
    duplicated_matrix = np.zeros((rows + 2, cols + 2), dtype=matrix.dtype)
    
    # Copie des valeurs de la matrice d'origine dans la nouvelle matrice
    duplicated_matrix[1:rows+1, 1:cols+1] = matrix
    
    # Duplication des bords horizontaux
    duplicated_matrix[0, 1:cols+1] = matrix[0, :]
    duplicated_matrix[rows+1, 1:cols+1] = matrix[rows-1, :]
    
    # Duplication des bords verticaux
    duplicated_matrix[1:rows+1, 0] = matrix[:, 0]
    duplicated_matrix[1:rows+1, cols+1] = matrix[:, cols-1]
    
    # Duplication des coins
    duplicated_matrix[0, 0] = matrix[0, 0]
    duplicated_matrix[0, cols+1] = matrix[0, cols-1]
    duplicated_matrix[rows+1, 0] = matrix[rows-1, 0]
    duplicated_matrix[rows+1, cols+1] = matrix[rows-1, cols-1]
    
    return duplicated_matrix

# Exemple d'utilisation
original_matrix = np.array([
    [1,  2,  3,  4,  5,  6,  7,  8],
    [9,  10, 11, 12, 13, 14, 15, 16],
    [17, 18, 19, 20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29, 30, 31, 32],
    [33, 34, 35, 36, 37, 38, 39, 40],
    [41, 42, 43, 44, 45, 46, 47, 48],
    [49, 50, 51, 52, 53, 54, 55, 56],
    [57, 58, 59, 60, 61, 62, 63, 64]
])

duplicated_matrix = duplicate_border(original_matrix)

In [14]:
duplicated_matrix

array([[ 1,  1,  2,  3,  4,  5,  6,  7,  8,  8],
       [ 1,  1,  2,  3,  4,  5,  6,  7,  8,  8],
       [ 9,  9, 10, 11, 12, 13, 14, 15, 16, 16],
       [17, 17, 18, 19, 20, 21, 22, 23, 24, 24],
       [25, 25, 26, 27, 28, 29, 30, 31, 32, 32],
       [33, 33, 34, 35, 36, 37, 38, 39, 40, 40],
       [41, 41, 42, 43, 44, 45, 46, 47, 48, 48],
       [49, 49, 50, 51, 52, 53, 54, 55, 56, 56],
       [57, 57, 58, 59, 60, 61, 62, 63, 64, 64],
       [57, 57, 58, 59, 60, 61, 62, 63, 64, 64]])

In [39]:
import numpy as np

def duplicate_border(matrix, border_size):
    # Récupération des dimensions de la matrice d'origine
    rows, cols = matrix.shape
    
    # Création d'une nouvelle matrice avec des dimensions augmentées pour accueillir les bords dupliqués
    duplicated_rows = rows + 2 * border_size
    duplicated_cols = cols + 2 * border_size
    duplicated_matrix = np.zeros((duplicated_rows, duplicated_cols), dtype=matrix.dtype)
    
    # Copie des valeurs de la matrice d'origine dans la nouvelle matrice
    duplicated_matrix[border_size:border_size+rows, border_size:border_size+cols] = matrix
    
    # Duplication des bords horizontaux
    duplicated_matrix[:border_size, border_size:border_size+cols] = matrix[0, :]  # Bord supérieur
    duplicated_matrix[border_size+rows: border_size*2 + rows, border_size:border_size+cols] = matrix[-1, :]  # Bord inférieur
    
    # Duplication des bords verticaux
    duplicated_matrix[border_size:border_size+rows, :border_size] = matrix[:, 0].reshape(-1, 1)  # Bord gauche
    duplicated_matrix[border_size:border_size+rows, border_size+cols:border_size*2+cols] = matrix[:, -1].reshape(-1, 1)  # Bord droit
    
    # Duplication des coins
    duplicated_matrix[:border_size, :border_size] = matrix[0, 0]  # Coin supérieur gauche
    duplicated_matrix[:border_size, border_size+cols:border_size*2+cols] = matrix[0, -1]  # Coin supérieur droit
    duplicated_matrix[border_size+rows: border_size*2+rows, :border_size] = matrix[-1, 0]  # Coin inférieur gauche
    duplicated_matrix[border_size+rows: border_size*2+rows, border_size+cols:border_size*2+cols] = matrix[-1, -1]  # Coin inférieur droit
    
    return duplicated_matrix

# Exemple d'utilisation
original_matrix = np.array([
    [1,  2,  3,  4,  5,  6,  7,  8],
    [9,  10, 11, 12, 13, 14, 15, 16],
    [17, 18, 19, 20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29, 30, 31, 32],
    [33, 34, 35, 36, 37, 38, 39, 40],
    [41, 42, 43, 44, 45, 46, 47, 48],
    [49, 50, 51, 52, 53, 54, 55, 56],
    [57, 58, 59, 60, 61, 62, 63, 64]
])

border_size = 4  # Taille supplémentaire pour les bords

duplicated_matrix = duplicate_border(original_matrix, border_size)


In [40]:
duplicated_matrix

array([[ 1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,  8],
       [ 1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,  8],
       [ 1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,  8],
       [ 1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,  8],
       [ 1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,  8],
       [ 9,  9,  9,  9,  9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 16, 16],
       [17, 17, 17, 17, 17, 18, 19, 20, 21, 22, 23, 24, 24, 24, 24, 24],
       [25, 25, 25, 25, 25, 26, 27, 28, 29, 30, 31, 32, 32, 32, 32, 32],
       [33, 33, 33, 33, 33, 34, 35, 36, 37, 38, 39, 40, 40, 40, 40, 40],
       [41, 41, 41, 41, 41, 42, 43, 44, 45, 46, 47, 48, 48, 48, 48, 48],
       [49, 49, 49, 49, 49, 50, 51, 52, 53, 54, 55, 56, 56, 56, 56, 56],
       [57, 57, 57, 57, 57, 58, 59, 60, 61, 62, 63, 64, 64, 64, 64, 64],
       [57, 57, 57, 57, 57, 58, 59, 60, 61, 62, 63, 64, 64, 64, 64, 64],
       [57, 57, 57, 57, 57, 58, 59, 60, 61, 62, 63,

In [66]:
def wrap_border(matrix):
    # Dimensions de la matrice
    rows = len(matrix)
    cols = len(matrix[0])

    # Matrice résultante avec une taille supplémentaire pour les bords répétés
    wrapped_matrix = [[0] * (cols + 2) for _ in range(rows + 2)]

    # Copie des valeurs de la matrice d'origine au centre de la matrice résultante
    for i in range(rows):
        for j in range(cols):
            wrapped_matrix[i + 1][j + 1] = matrix[i][j]

    # Répétition des bords de la matrice d'origine
    for i in range(rows):
        wrapped_matrix[i + 1][0] = matrix[i][-1]  # Copie de la dernière colonne à gauche de la première colonne
        wrapped_matrix[i + 1][-1] = matrix[i][0]  # Copie de la première colonne à droite de la dernière colonne

    # Répétition des bords supérieur et inférieur de la matrice d'origine
    wrapped_matrix[0] = wrapped_matrix[-2]  # Copie de la dernière ligne en haut de la première ligne
    wrapped_matrix[-1] = wrapped_matrix[1]  # Copie de la première ligne en bas de la dernière ligne

    return np.array(wrapped_matrix)

# Matrice d'exemple 8x8
matrix = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
       [12, 12, 14, 19, 26, 58, 60, 55],
       [14, 13, 16, 24, 40, 57, 69, 56],
       [14, 17, 22, 29, 51, 87, 80, 62],
       [18, 22, 37, 56, 68, 109, 103, 77],
       [24, 35, 55, 64, 81, 104, 113, 92],
       [49, 64, 78, 87, 103, 121, 120, 101],
       [72, 92, 95, 98, 112, 100, 103, 99]
])

# Application de l'effet de bord d'enroulement
result1 = wrap_border(matrix)

In [67]:
result1

array([[64, 57, 58, 59, 60, 61, 62, 63, 64, 57],
       [ 8,  1,  2,  3,  4,  5,  6,  7,  8,  1],
       [16,  9, 10, 11, 12, 13, 14, 15, 16,  9],
       [24, 17, 18, 19, 20, 21, 22, 23, 24, 17],
       [32, 25, 26, 27, 28, 29, 30, 31, 32, 25],
       [40, 33, 34, 35, 36, 37, 38, 39, 40, 33],
       [48, 41, 42, 43, 44, 45, 46, 47, 48, 41],
       [56, 49, 50, 51, 52, 53, 54, 55, 56, 49],
       [64, 57, 58, 59, 60, 61, 62, 63, 64, 57],
       [ 8,  1,  2,  3,  4,  5,  6,  7,  8,  1]])

In [102]:
import numpy as np

def wrap_border(matrix, num_blocs):
    if num_blocs == 16:
      border_size = int(matrix.shape[0] / 2)
      # Dimensions de la matrice d'origine
      rows, cols = matrix.shape

      # Matrice résultante avec une taille supplémentaire pour les bords répétés
      wrapped_rows = rows + 2 * border_size
      wrapped_cols = cols + 2 * border_size
      wrapped_matrix = np.zeros((wrapped_rows, wrapped_cols), dtype=matrix.dtype)

      # Copie des valeurs de la matrice d'origine au centre de la matrice résultante
      wrapped_matrix[border_size:border_size + rows, border_size:border_size + cols] = matrix

      # Répétition des bords de la matrice d'origine
      wrapped_matrix[:border_size, border_size:border_size + cols] = matrix[-border_size:, :]  # Bord supérieur
      wrapped_matrix[-border_size:, border_size:border_size + cols] = matrix[:border_size, :]  # Bord inférieur
      wrapped_matrix[border_size:border_size + rows, :border_size] = matrix[:, -border_size:]  # Bord gauche
      wrapped_matrix[border_size:border_size + rows, -border_size:] = matrix[:, :border_size]  # Bord droit

      # Coins
      wrapped_matrix[:border_size, :border_size] = matrix[-border_size:, -border_size:]  # Coin supérieur gauche
      wrapped_matrix[:border_size, -border_size:] = matrix[-border_size:, :border_size]   # Coin supérieur droit
      wrapped_matrix[-border_size:, :border_size] = matrix[:border_size, -border_size:]   # Coin inférieur gauche
      wrapped_matrix[-border_size:, -border_size:] = matrix[:border_size, :border_size]    # Coin inférieur droit

      return wrapped_matrix
    else:
      return matrix

# Matrice d'exemple 8x8
#matrix = np.random.randint(0, 256, size=(8, 8))
matrix = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
       [12, 12, 14, 19, 26, 58, 60, 55],
       [14, 13, 16, 24, 40, 57, 69, 56],
       [14, 17, 22, 29, 51, 87, 80, 62],
       [18, 22, 37, 56, 68, 109, 103, 77],
       [24, 35, 55, 64, 81, 104, 113, 92],
       [49, 64, 78, 87, 103, 121, 120, 101],
       [72, 92, 95, 98, 112, 100, 103, 99]
])

# Application de l'effet de bord d'enroulement avec la taille de bordure spécifiée
result = wrap_border(matrix, 8)

In [107]:
import numpy as np

def unwrap_border(wrapped_matrix, num_blocs):
    if num_blocs == 16:
        # Vérification que la matrice a la bonne taille
        if len(wrapped_matrix) != 16 or len(wrapped_matrix[0]) != 16:
            raise ValueError("La matrice doit être de taille 16x16")
        
        # Création de la matrice résultante de taille 8x8
        matrice_reduite = [[0] * 8 for _ in range(8)]
        
        # Parcours de la matrice 16x16 et copie des éléments pertinents dans la matrice réduite
        for i in range(4, 12):
            for j in range(4, 12):
                matrice_reduite[i-4][j-4] = wrapped_matrix[i][j]
        
        return matrice_reduite
    else:
        return wrapped_matrix

# Création d'une matrice de test
matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                  [12, 12, 14, 19, 26, 58, 60, 55],
                                  [14, 13, 16, 24, 40, 57, 69, 56],
                                  [14, 17, 22, 29, 51, 87, 80, 62],
                                  [18, 22, 37, 56, 68, 109, 103, 77],
                                  [24, 35, 55, 64, 81, 104, 113, 92],
                                  [49, 64, 78, 87, 103, 121, 120, 101],
                                  [72, 92, 95, 98, 112, 100, 103, 99]])

# Envelopper les bords de la matrice
wrapped_matrix = wrap_border(matrix, 16)
print("Matrice enveloppée :\n", wrapped_matrix)

# Restaurer la matrice d'origine
restored_matrix = unwrap_border(wrapped_matrix, 16)
print("\nMatrice restaurée :\n", restored_matrix)
print((matrix == restored_matrix).all())

Matrice enveloppée :
 [[ 68 109 103  77  18  22  37  56  68 109 103  77  18  22  37  56]
 [ 81 104 113  92  24  35  55  64  81 104 113  92  24  35  55  64]
 [103 121 120 101  49  64  78  87 103 121 120 101  49  64  78  87]
 [112 100 103  99  72  92  95  98 112 100 103  99  72  92  95  98]
 [ 24  40  51  61  16  11  10  16  24  40  51  61  16  11  10  16]
 [ 26  58  60  55  12  12  14  19  26  58  60  55  12  12  14  19]
 [ 40  57  69  56  14  13  16  24  40  57  69  56  14  13  16  24]
 [ 51  87  80  62  14  17  22  29  51  87  80  62  14  17  22  29]
 [ 68 109 103  77  18  22  37  56  68 109 103  77  18  22  37  56]
 [ 81 104 113  92  24  35  55  64  81 104 113  92  24  35  55  64]
 [103 121 120 101  49  64  78  87 103 121 120 101  49  64  78  87]
 [112 100 103  99  72  92  95  98 112 100 103  99  72  92  95  98]
 [ 24  40  51  61  16  11  10  16  24  40  51  61  16  11  10  16]
 [ 26  58  60  55  12  12  14  19  26  58  60  55  12  12  14  19]
 [ 40  57  69  56  14  13  16  24  40  5

In [127]:
import numpy as np

def quantize_vector(vector, num_intervals):
    min_val = min(vector)
    max_val = max(vector)
    interval_size = (max_val - min_val) / num_intervals
    quantized_vector = np.floor((vector - min_val) / interval_size)
    return quantized_vector, min_val, interval_size

def dequantize_vector(quantized_vector, min_val, interval_size):
    return quantized_vector * interval_size + min_val

def ext_chr(chaine):
    occ = {c: chaine.count(c) for c in set(chaine)}
    return sorted(occ.items(), key=lambda x: x[1], reverse=True)

def huffman_dictionnary_code(node, binString=''):
    if isinstance(node, str):
        return {node: binString}
    (l, r) = node
    d = {}
    d.update(huffman_dictionnary_code(l, binString + '0'))
    d.update(huffman_dictionnary_code(r, binString + '1'))
    return d

def huffman_tree(chaine):
    nodes = ext_chr(chaine)
    
    while len(nodes) > 1:
        (key1, c1) = nodes.pop()
        (key2, c2) = nodes.pop()
        node = (key1, key2)
        nodes.append((node, c1 + c2))
        nodes.sort(key=lambda x: x[1], reverse=True)
        
    return nodes[0][0]

def encoding_huffman(strings):
    huffman_tree_result = huffman_tree(strings)  # Stockez l'arbre de Huffman dans une variable distincte
    huffmanCode = huffman_dictionnary_code(huffman_tree_result)
    compressed_string = ''
    
    for char in strings:
        compressed_string += huffmanCode[char]
        
    return compressed_string, huffman_tree_result

def decoding_huffman(compressed_string, huffman_tree):
    decoded_string = ''
    current_node = huffman_tree
    
    for bit in compressed_string:
        if bit == '0':
            current_node = current_node[0]
        else:
            current_node = current_node[1]
        
        if isinstance(current_node, str):
            decoded_string += current_node
            current_node = huffman_tree
    
    return decoded_string

# Exemple de vecteur de valeurs float64
vector = np.array([-3.5, 2.0, 1.5, -4.0, 3.0, -2.5])

# Paramètres de quantification
num_intervals = 1000

# Quantification du vecteur
quantized_vector, min_val, interval_size = quantize_vector(vector, num_intervals)
dequantized_vector = dequantize_vector(quantized_vector, min_val, interval_size)

print("Vecteur original :", vector)
print("Vecteur quantifié :", quantized_vector)
print("Vecteur dequantifié :", dequantized_vector)

# Convertir le vecteur quantifié en chaîne de caractères
quantized_string = ''.join(map(str, quantized_vector))

# Appliquer l'algorithme de Huffman
compressed_string, huffman_tree_result = encoding_huffman(quantized_string)

# Imprimer la chaîne compressée
print("Chaîne compressée :", compressed_string)

Vecteur original : [-3.5  2.   1.5 -4.   3.  -2.5]
Vecteur quantifié : [  71.  857.  785.    0. 1000.  214.]
Vecteur dequantifié : [-3.503  1.999  1.495 -4.     3.    -2.502]
Chaîne compressée : 10000101110001011100011110000010110111110111001111111011110101001101000111


Chaîne de caractères: -1236780
Tailles des éléments int64: [3, 3, 1]
Vecteur reconstruit: [-12. 367.   8.]


In [160]:
import numpy as np

# Génération d'un vecteur aléatoire de taille 64 avec des valeurs float64
vector = np.random.rand(64) * 100 - 50

In [172]:
def list_to_string(lst):
    # Convertit chaque nombre en chaîne de caractères et les joint avec des virgules
    string = ''.join(str(num) for num in lst.tolist())
    lengths = [len(str(i)) for i in lst.tolist()]
    
    return string, lengths

def string_to_list(string, lengths):
    start = 0
    lst = []
    for length in lengths:
        num_str = string[start:start+length]
        lst.append(int(num_str))
        start += length
    return lst

# Exemple d'utilisation
liste = np.array([1, -2, 3, -4, 5])
chaine, lengths = list_to_string(liste)
print("Liste transformée en chaîne:", chaine)

nouvelle_liste = string_to_list(chaine, lengths)
print("Chaîne transformée en liste:", nouvelle_liste)
print((liste == nouvelle_liste).all())

Liste transformée en chaîne: 1-23-45
Chaîne transformée en liste: [1, -2, 3, -4, 5]
True
